In [0]:
%run vis/xAPISG-noDataToFillVisualization.ipynb # notebook to create the visualisation with a message NoDataToFill
%run vis/helpersFunctions/OrderGraphValuesAndSelectDataByXaxis.ipynb

Selectors :
   * orderItemBarChart : dropdown value indicating how to order xlabel : alphabetically or by y value (hight to low or low to hight)

In [ ]:
#constructing a dropdown to order for alphabetic keys or for values
orderItemBarChart=widgets.Dropdown(options=[('alphabetic',0),('value from highest to lowest',1),('value from lowest to highest',-1)],
                               description='Order',
                               disabled=False)

***vis_items_interacted_times_interact* function** :
 * Output : the widgets selector and the outputBarChartItem in a list to be display in a VBox
 * outputBarChartItem : displays bar chart showing for each item the number of use by players

In [ ]:
def vis_items_interacted_times_interact():
    def display_item_bar_chart(change):
        with outputBarChartItem:
            clear_output(wait=False)
            vis_items_interacted_times()
            clear_output(wait=True)
    orderItemBarChart.observe(display_item_bar_chart, 'value')
    display_item_bar_chart(None)
    return [orderItemBarChart, outputBarChartItemUpdate, outputBarChartItem]

***vis_items_interacted_times_heatMap_interacted* function** :
* Output : the outputHeatMapItem in a list to be display in a VBox
* outputHeatMapItem: displays heatMap showing for each item the number of use by players

In [ ]:
def vis_items_interacted_times_heatmap_interacted():
    def display_item_heatmap(change):
        with outputHeatMapItem:
            clear_output(wait=False)
            vis_items_interacted_times_heatMap()
            clear_output(wait=True)
    display_item_heatmap(None)
    return [outputHeatMapItemUpdate, outputHeatMapItem]

***vis_items_interacted_function_time_interacted* function** :
* Output : the outputBubbleChartItem in a list to be display in a VBox
* outputBubbleChartItem: display a bubble chart showing a bubble in function of the average of players has interacted with the item in a certain period of time.

In [ ]:
def vis_items_interacted_function_time_interacted():
    def display_item_bubble_chart(change):
        with outputBubbleChartItem:
            clear_output(wait=False)
            vis_items_interacted_function_time()
            clear_output(wait=True)
    display_item_bubble_chart(None)
    return [outputBubbleChartItemUpdate, outputBubbleChartItem]

***vis_items_interacted_times* function** displays bar chart showing for each item the number of use by players

In [2]:
%run vis/helpersFunctions/MultiBarSeparated.ipynb
def vis_items_interacted_times():
    #get all players and items selected
    items=getSublistSelectedFromUnionListSelected(
        item_interaction_list,
        checkboxesItemInteractedSelected.get_selected_options()
    )
    playersSelected=checkboxesPlayersSelected.get_selected_options()
    if len(items)>0 and len(playersSelected)>0:
        accordionBarChartItem=widgets.Accordion(children=[widgets.Output() for i in items],
                                                selected_index = None)
        for i in range(len(items)):
            accordionBarChartItem.set_title(i, items[i])
        display(accordionBarChartItem)
        def on_selected_index_change_item_bar_chart(change):
            #close and clear previous fig of items interactions
            for fig in figBarChartItem:
                plt.close(fig=fig)
            figBarChartItem.clear()
            index=change['new']
            if index!=None:
                #get all players and items selected
                items=getSublistSelectedFromUnionListSelected(
                    item_interaction_list,
                    checkboxesItemInteractedSelected.get_selected_options()
                )
                item=items[index]
                playersSelected=checkboxesPlayersSelected.get_selected_options()
                nbitem=constructSimpleTab(playersSelected, item, "interactions", True)
                nbitem=orderDict(nbitem,orderItemBarChart.value)
                # Simple bar chart showing the item selected for each person.
                with accordionBarChartItem.children[index]:
                    clear_output(wait=False)
                    #create new fig with custom name in the tab
                    name=os.path.splitext(filename)[0]+"_interaction_with_item_"+item
                    fig = plt.figure(num=name, constrained_layout=True)
                    fig.canvas.header_visible = False
                    fig.canvas.layout.min_height = '400px'
                    figBarChartItem.append(fig)
                    displayEachBar(nbitem, item, "Interaction with item ", "Players", "Numbers of interaction")
        accordionBarChartItem.observe(on_selected_index_change_item_bar_chart,'selected_index')
    else:
        #close and clear previous fig of items interactions
        for fig in figBarChartItem:
            plt.close(fig=fig)
        figBarChartItem.clear()
        #create new fig with custom name in the tab
        name=os.path.splitext(filename)[0]+"_interaction_with_item"
        fig = plt.figure(num=name, constrained_layout=True)
        fig.canvas.header_visible = False
        fig.canvas.layout.min_height = '400px'
        figBarChartItem.append(fig)
        noDataToFillVis(10)
        plt.title("Interaction with item")
        plt.xlabel("Players")
        plt.ylabel("Numbers of interaction")
        plt.show()
    with outputBarChartItemUpdate:
        clear_output(wait=False)

***vis_items_interacted_times_heatMap* function** displays heatMap showing for each item the number of use by players

In [ ]:
%run vis/helpersFunctions/HeatMap.ipynb
def vis_items_interacted_times_heatMap():
    #close and clear previous fig of items interactions
    for fig in figHeatMapItem:
        plt.close(fig=fig)
    figHeatMapItem.clear()
    #get all players and items selected
    label_players=checkboxesPlayersSelected.get_selected_options()
    label_items=getSublistSelectedFromUnionListSelected(item_interaction_list, checkboxesItemInteractedSelected.get_selected_options())
    nbitems=constructHeatMap(players_info, label_players, label_items, "interactions", True)
    ## Now use the heatmap function to plot the results
    if len(label_players)>0 and len(label_items)>0:
        heatMap(nbitems, label_players, label_items,
                figHeatMapItem, "_HeatMap_interaction_with_item_by_players")
    else:
        #create new fig with custom name in the tab
        name=os.path.splitext(filename)[0]+"_HeatMap_interaction_with_item_by_players"
        fig = plt.figure(num=name, constrained_layout=True)
        fig.canvas.header_visible = False
        fig.canvas.layout.min_height = '400px'
        figHeatMapItem.append(fig)
        noDataToFillVis(10)
    plt.xlabel("Players")
    plt.ylabel("Items")
    plt.title("Heatmap showing interaction with item by players")
    plt.show()
    with outputHeatMapItemUpdate:
        clear_output(wait=False)

***vis_items_interacted_function_time* function** display a bubble chart showing a bubble in function of the average of players has interacted with the item in a certain period of time.

In [ ]:
%run vis/helpersFunctions/bubbleChartFunctionTime.ipynb
def vis_items_interacted_function_time():
    #close and clear previous fig of items interactions
    for fig in figBubbleChartItem:
        plt.close(fig=fig)
    figBubbleChartItem.clear()
    #get all players and items selected
    playersSelected=checkboxesPlayersSelected.get_selected_options()
    items=getSublistSelectedFromUnionListSelected(item_interaction_list, checkboxesItemInteractedSelected.get_selected_options())
    players_times_interacted={}
    for item in items:
        for player in playersSelected:
            for type in players_info[player]["interactions"].keys():
                if item in players_info[player]["interactions"][type].keys():
                    for time in players_info[player]["interactions"][type][item]:
                        if item not in players_times_interacted.keys():
                            players_times_interacted[item]={}
                        players_times_interacted[item][time]=player
    #create new fig with custom name in the tab
    name=os.path.splitext(filename)[0]+"_bubbleChart_item_interacted_function_time_by_all_players"
    fig = plt.figure(num=name, constrained_layout=True)
    fig.canvas.header_visible = False
    fig.canvas.layout.min_height = '400px'
    figBubbleChartItem.append(fig)
    if len(items)>0 and len(playersSelected)>0:
        constructAndDisplayBubbleChart(items,players_times_interacted)
    else:
        noDataToFillVis(10)
    font=10
    plt.xlabel("Time", fontsize=font)
    plt.ylabel("Items", fontsize=font)
    plt.title("Item interacted in function of time by all players")
    plt.show()
    with outputBubbleChartItemUpdate:
        clear_output(wait=False)